In [1]:
import sympy as sp
import numpy as np
thp, thd, tp, td, lp, ld, alp, r, s = sp.symbols(r'\theta_p \theta_d t_p t_d l_p l_d \alpha r s', real = True)
kda = td / (ld * r)
kd1 = alp * kda
kd = 2 * (kda - kd1) * s / ld + kd1
display(kd)

\alpha*t_d/(l_d*r) + s*(-2*\alpha*t_d/(l_d*r) + 2*t_d/(l_d*r))/l_d

In [2]:
with sp.assuming(sp.Q.nonzero(ld)):
    dp = sp.integrate(r * kd, (s, 0, lp))
display(sp.simplify(dp))

l_p*t_d*(\alpha*l_d - l_p*(\alpha - 1))/l_d**2

프록시말텐던 중립상태에서 디스털만 구동 -- 프록시말 부분에서는 유효곡률은 그냥 곡률의 1/2 (r -> 2r)이 되는 효과

아니면 컴프레션 무시하는것도 좋을듯 그냥 dp만큼 줄고 늘었으니 dp만큼 카운터치면 되는거아님? >> 이러면 프록시말 길이변동은 없음

In [3]:
dpnew = dp / 2

In [4]:
# 회전행렬 테스트
mat = np.array([1,0])
theta = np.pi / 2
rot = np.array([[np.cos(theta), -np.sin(theta)],
                [np.sin(theta), np.cos(theta)]])
rotval = rot @ mat
display(rotval)

array([6.123234e-17, 1.000000e+00])

기존 듀얼벤딩 식을 재구성하면 어케되는가?
dp대신 lcst 사용
평면만 따질 것이므로 방향은 2개만 따지면 된다

teuler에서는 t_eq로 쓰긴 했는데 실질적으로 얘만 굽힘 형성에 관여하게 되므로 유효구동량으로 표현을 바꾸는 것도 좋은듯하다

In [5]:
lcst = lp + tp * (sp.sign(tp)*sp.sign(td))
display(lcst)

l_p + t_p*sign(t_d)*sign(t_p)

논문꺼드럭용 일반화

In [6]:
tpeff = sp.Symbol(r'\Delta t_{p,eff}', real=True)
tdeff = sp.Symbol(r'\Delta t_{d,eff}', real=True)
phip = sp.Symbol(r'\phi_{p,eff}', real=True)
phid = sp.Symbol(r'\phi_{d,eff}', real=True)
display(tpeff + tdeff + phip + phid)

\Delta t_{d,eff} + \Delta t_{p,eff} + \phi_{d,eff} + \phi_{p,eff}

In [17]:
lcstraw = lp + tpeff * sp.cos(phip - phid)
cossym = sp.Symbol(r'c_{\phi}')
lcst = lp + tpeff * cossym
kda2 = tdeff / (ld * r)
kd12 = kda2 * alp
kd2 = 2 * (kda2 - kd12) * s / ld + kd12
display(kd2)
display(sp.simplify(kd2))
print(sp.latex(sp.simplify(kd2)))
lcstsym = sp.Symbol(r'l_{cst}')
with sp.assuming(sp.Q.nonzero(ld)):
    dpnew = sp.integrate(r * kd2 / 2, (s, 0, lcst))
    dpnew_lcstsym = sp.integrate(r * kd2 / 2, (s, 0, lcstsym))
display(dpnew)
display(dpnew_lcstsym)
print(sp.latex(lcst))
print(sp.latex(sp.simplify(kd2)))
print(sp.latex(dpnew_lcstsym))

\Delta t_{d,eff}*\alpha/(l_d*r) + s*(-2*\Delta t_{d,eff}*\alpha/(l_d*r) + 2*\Delta t_{d,eff}/(l_d*r))/l_d

\Delta t_{d,eff}*(\alpha*l_d - 2*s*(\alpha - 1))/(l_d**2*r)

\frac{\Delta t_{d,eff} \left(\alpha l_{d} - 2 s \left(\alpha - 1\right)\right)}{l_{d}^{2} r}


\Delta t_{d,eff}*\alpha*(\Delta t_{p,eff}*c_{\phi} + l_p)/(2*l_d) + (-\Delta t_{d,eff}*\alpha + \Delta t_{d,eff})*(\Delta t_{p,eff}*c_{\phi} + l_p)**2/(2*l_d**2)

\Delta t_{d,eff}*\alpha*l_{cst}/(2*l_d) + l_{cst}**2*(-\Delta t_{d,eff}*\alpha + \Delta t_{d,eff})/(2*l_d**2)

\Delta t_{p,eff} c_{\phi} + l_{p}
\frac{\Delta t_{d,eff} \left(\alpha l_{d} - 2 s \left(\alpha - 1\right)\right)}{l_{d}^{2} r}
\frac{\Delta t_{d,eff} \alpha l_{cst}}{2 l_{d}} + \frac{l_{cst}^{2} \left(- \Delta t_{d,eff} \alpha + \Delta t_{d,eff}\right)}{2 l_{d}^{2}}


In [12]:
tpeqn = sp.Eq(tp, tpeff - dpnew_lcstsym)
tdeqn = sp.Eq(td, tpeff + tdeff + dpnew_lcstsym)
display(tpeqn)
display(tdeqn)

sol = sp.solve([tpeqn, tdeqn],(tpeff, tdeff))
display(sol[tpeff])
display(sol[tdeff])

tpeqn2 = sp.Eq(tp, tpeff - dpnew)
tdeqn2 = sp.Eq(td, tpeff + tdeff + dpnew)
sol2 = sp.solve([tpeqn2, tdeqn2],(tp,td))
display(sp.simplify(sol2[tp]))
display(sol2[td])
display(tpeqn2)
display(tdeqn2)
print(sp.latex(sp.simplify(tpeqn2)))
print(sp.latex(tdeqn2))

Eq(t_p, -\Delta t_{d,eff}*\alpha*l_{cst}/(2*l_d) + \Delta t_{p,eff} - l_{cst}**2*(-\Delta t_{d,eff}*\alpha + \Delta t_{d,eff})/(2*l_d**2))

Eq(t_d, \Delta t_{d,eff}*\alpha*l_{cst}/(2*l_d) + \Delta t_{d,eff} + \Delta t_{p,eff} + l_{cst}**2*(-\Delta t_{d,eff}*\alpha + \Delta t_{d,eff})/(2*l_d**2))

(\alpha*l_d*l_{cst}*t_d + \alpha*l_d*l_{cst}*t_p - \alpha*l_{cst}**2*t_d - \alpha*l_{cst}**2*t_p + 2*l_d**2*t_p + l_{cst}**2*t_d + l_{cst}**2*t_p)/(2*\alpha*l_d*l_{cst} - 2*\alpha*l_{cst}**2 + 2*l_d**2 + 2*l_{cst}**2)

(l_d**2*t_d - l_d**2*t_p)/(\alpha*l_d*l_{cst} - \alpha*l_{cst}**2 + l_d**2 + l_{cst}**2)

(\Delta t_{d,eff}*\Delta t_{p,eff}**2*\alpha*c_{\phi}**2 - \Delta t_{d,eff}*\Delta t_{p,eff}**2*c_{\phi}**2 - \Delta t_{d,eff}*\Delta t_{p,eff}*\alpha*c_{\phi}*l_d + 2*\Delta t_{d,eff}*\Delta t_{p,eff}*\alpha*c_{\phi}*l_p - 2*\Delta t_{d,eff}*\Delta t_{p,eff}*c_{\phi}*l_p - \Delta t_{d,eff}*\alpha*l_d*l_p + \Delta t_{d,eff}*\alpha*l_p**2 - \Delta t_{d,eff}*l_p**2 + 2*\Delta t_{p,eff}*l_d**2)/(2*l_d**2)

(-\Delta t_{d,eff}*\Delta t_{p,eff}**2*\alpha*c_{\phi}**2 + \Delta t_{d,eff}*\Delta t_{p,eff}**2*c_{\phi}**2 + \Delta t_{d,eff}*\Delta t_{p,eff}*\alpha*c_{\phi}*l_d - 2*\Delta t_{d,eff}*\Delta t_{p,eff}*\alpha*c_{\phi}*l_p + 2*\Delta t_{d,eff}*\Delta t_{p,eff}*c_{\phi}*l_p + \Delta t_{d,eff}*\alpha*l_d*l_p - \Delta t_{d,eff}*\alpha*l_p**2 + 2*\Delta t_{d,eff}*l_d**2 + \Delta t_{d,eff}*l_p**2 + 2*\Delta t_{p,eff}*l_d**2)/(2*l_d**2)

Eq(t_p, -\Delta t_{d,eff}*\alpha*(\Delta t_{p,eff}*c_{\phi} + l_p)/(2*l_d) + \Delta t_{p,eff} - (-\Delta t_{d,eff}*\alpha + \Delta t_{d,eff})*(\Delta t_{p,eff}*c_{\phi} + l_p)**2/(2*l_d**2))

Eq(t_d, \Delta t_{d,eff}*\alpha*(\Delta t_{p,eff}*c_{\phi} + l_p)/(2*l_d) + \Delta t_{d,eff} + \Delta t_{p,eff} + (-\Delta t_{d,eff}*\alpha + \Delta t_{d,eff})*(\Delta t_{p,eff}*c_{\phi} + l_p)**2/(2*l_d**2))

t_{p} = \frac{- \Delta t_{d,eff} \alpha l_{d} \left(\Delta t_{p,eff} c_{\phi} + l_{p}\right) + \Delta t_{d,eff} \left(\alpha - 1\right) \left(\Delta t_{p,eff} c_{\phi} + l_{p}\right)^{2} + 2 \Delta t_{p,eff} l_{d}^{2}}{2 l_{d}^{2}}
t_{d} = \frac{\Delta t_{d,eff} \alpha \left(\Delta t_{p,eff} c_{\phi} + l_{p}\right)}{2 l_{d}} + \Delta t_{d,eff} + \Delta t_{p,eff} + \frac{\left(- \Delta t_{d,eff} \alpha + \Delta t_{d,eff}\right) \left(\Delta t_{p,eff} c_{\phi} + l_{p}\right)^{2}}{2 l_{d}^{2}}
